# Data quality checks

## Setup

In [1]:
import pandas as pd

In [2]:
import random

## Dimensions of Data Quality

### Timeliness

In [3]:
random.seed(42)
n_sensors = 20
locations = ["Villach", "Klagenfurt", "Spittal", "Feldkirchen"]
mean_sensor_delay = dict(zip(locations, [1, 1, 10, 4]))

pd.DataFrame(
    {
        "sensor_location": [
            random.choice(locations)
            for _ in range(n_sensors)
        ],
        # Usually, you would use a data type specific to time
        # in the following two columns.
        "time_of_measurement": [
            round(random.random() * 10_000, 2)
            for _ in range(n_sensors)
        ],
    }
).assign(
    time_of_ingestion=lambda df:
        df.time_of_measurement
        + df.sensor_location.map(mean_sensor_delay) + 2 * random.random()
)

,sensor_location,time_of_measurement,time_of_ingestion
0,Villach,8094.30,8097.023414
1,Villach,64.99,67.713414
2,Spittal,8058.19,8069.913414
3,Klagenfurt,6981.39,6984.113414
4,Klagenfurt,3402.51,3405.233414
5,Klagenfurt,1554.79,1557.513414
6,Villach,9572.13,9574.853414
7,Villach,3365.95,3368.673414
8,Feldkirchen,927.46,933.183414
9,Villach,967.16,969.883414


Which location has the largest delay between measurement and ingestion?

### Completeness

#### a)

Similar to timeliness with missing values instead of delays. (For instance: how many missing values are there in each city? Also possible: how many values are NA (not available) in each column?)

#### b)

In [4]:
n_items = 20

item_df = pd.DataFrame(
    {
        "item_number":
            pd.Series(range(n_items))
            .map(lambda n: f"product_category_A__item_number_{n}"),
        "price":
            pd.Series(random.random() for _ in range(n_items)).round(2)
    }
).drop([9, 17]).reset_index()

In [5]:
item_df

,index,item_number,price
0,0,product_category_A__item_number_0,0.58
1,1,product_category_A__item_number_1,0.70
2,2,product_category_A__item_number_2,0.05
3,3,product_category_A__item_number_3,0.23
4,4,product_category_A__item_number_4,0.29
5,5,product_category_A__item_number_5,0.08
6,6,product_category_A__item_number_6,0.23
7,7,product_category_A__item_number_7,0.10
8,8,product_category_A__item_number_8,0.28
9,10,product_category_A__item_number_10,0.36


Which item numbers seem to be missing?

### Consistency

#### a)

In [6]:
categories = list("ABBACABbACCaBACB")
random.seed(42)

pd.DataFrame(
    {
        "product_id": range(1, len(categories) + 1),
        "product_category": categories,
        "price": [round(random.random() * 100, 2)
                  for _ in range(len(categories))]
    }
)

,product_id,product_category,price
0,1,A,63.94
1,2,B,2.50
2,3,B,27.50
3,4,A,22.32
4,5,C,73.65
5,6,A,67.67
6,7,B,89.22
7,8,b,8.69
8,9,A,42.19
9,10,C,2.98


Programmatically determine which entries might be lacking consistency! Make sure that all values of product category are uppercase! (Do not fix these values one-by-one. Instead fix the problem for all wrong values at once.)

#### b)

In [7]:
categories = list("ABBACABBACCABACB")
list_len_1 = len(categories) // 3
list_len_2 = len(categories) - list_len_1
categories_1 = categories[:list_len_1] + [pd.NA for _ in range(list_len_2)]
categories_2 = [pd.NA for _ in range(list_len_1)] + categories[list_len_1:]
random.seed(42)

pd.DataFrame(
    {
        "product_category": categories_1,
        "Product Category": categories_2,
        "price": [round(random.random() * 100, 2)
                  for _ in range(len(categories))]
    }
)

,product_category,Product Category,price
0,A,<NA>,63.94
1,B,<NA>,2.50
2,B,<NA>,27.50
3,A,<NA>,22.32
4,C,<NA>,73.65
5,<NA>,A,67.67
6,<NA>,B,89.22
7,<NA>,B,8.69
8,<NA>,A,42.19
9,<NA>,C,2.98


#### c)

In [8]:
n_items = 20
random.seed(42)

pd.DataFrame(
    {
        "item_id": range(n_items),
        "price": [
            round(random.random() * 100, 2)
            for item_number in range(n_items)]
    }
).assign(price=lambda df: df.price.map(
        lambda price: f"{price}$" if random.random() > 0.7 else f"{price}€"
    )
).assign(price=lambda df: df.price.map(
        lambda price: price[-1] + price[:-2] if random.random() < 0.2 else price
))

,item_id,price
0,0,63.94$
1,1,€2.
2,2,27.5€
3,3,22.32€
4,4,$73.6
5,5,67.67€
6,6,€89.2
7,7,8.69€
8,8,42.19$
9,9,2.98€


Can you fix the inconsistency in the price column?

#### d)

Join (concatenate) the following two data frames in a meaningful way.

In [9]:
categories = list("ABBACABBACCABACB")
list_len_1 = len(categories) // 3
list_len_2 = len(categories) - list_len_1
categories_1 = categories[:list_len_1]
categories_2 = categories[list_len_1:]
random.seed(42)

df_1 = pd.DataFrame(
    {
        "product_category": categories_1,
        "price": [round(random.random() * 100, 2)
                  for _ in range(len(categories_1))]
    }
)
df_2 = pd.DataFrame(
    {
        "Product Category": categories_2,
        "price": [round(random.random() * 100, 2)
                  for _ in range(len(categories_2))]
    }
)

In [10]:
df_1

,product_category,price
0,A,63.94
1,B,2.50
2,B,27.50
3,A,22.32
4,C,73.65


In [11]:
df_2

,Product Category,price
0,A,67.67
1,B,89.22
2,B,8.69
3,A,42.19
4,C,2.98
5,C,21.86
6,A,50.54
7,B,2.65
8,A,19.88
9,C,64.99


### Uniqueness

In [12]:
pd.DataFrame(
    {
        "customer": ["Aleks", "Anita", "Markus", "Aleks", "Sigfried", "Stefan", "Tamara", "Ursula", "Markus"],
        "email": ["aleks@fh-kaernten.at",
                  "anita@fh-kaernten.at",
                  "markus@fh-kaernten.at",
                  "a@fh-kaernten.at",
                  "s@fh-kaernten.at",
                  "s@fh-kaernten.at",
                  "tamara@fh-kaernten.at",
                  "u@fh-kaernten.at",
                  "m@fh-kaernten.at"]
    },
    index=[0, 1, 2, 0, 3, 4, 5, 6, 7]
)

,customer,email
0,Aleks,aleks@fh-kaernten.at
1,Anita,anita@fh-kaernten.at
2,Markus,markus@fh-kaernten.at
0,Aleks,a@fh-kaernten.at
3,Sigfried,s@fh-kaernten.at
4,Stefan,s@fh-kaernten.at
5,Tamara,tamara@fh-kaernten.at
6,Ursula,u@fh-kaernten.at
7,Markus,m@fh-kaernten.at


Which problems related to uniqueness can you identify?

### Accuracy

In [13]:
pd.DataFrame(
    {
        "event_category": ["Sports", "Election", "Natural Disaster", "Sports", "Sports", "Awards", "Sports"],
        "lon": [-120, 0, 15, 350, -135, -200, 100],
        "lat": [50, 10, 45, 120, 60, -30, 20]
    }
)

,event_category,lon,lat
0,Sports,-120,50
1,Election,0,10
2,Natural Disaster,15,45
3,Sports,350,120
4,Sports,-135,60
5,Awards,-200,-30
6,Sports,100,20


Let's assume that each longitude must lie between -180 and 180 and each latitude must lie between -90 and 90. Find all lines (programmatically) where these assumptions are violated!